In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import pandas as pd
from PIL import Image
from torchsummary import summary
from torch.cuda.amp import GradScaler, autocast

In [2]:
allocated_memory = torch.cuda.memory_allocated()
print("Currently allocated GPU memory:", allocated_memory / 1024**2, "MB")

# Get peak memory usage
peak_memory = torch.cuda.max_memory_allocated()
print("Peak GPU memory usage:", peak_memory / 1024**2, "MB")

Currently allocated GPU memory: 0.0 MB
Peak GPU memory usage: 0.0 MB


In [3]:
num_gpus = torch.cuda.device_count()

# Get information about each GPU
for i in range(num_gpus):
    properties = torch.cuda.get_device_properties(i)
    print("GPU", i+1, "Total Memory:", properties.total_memory / (1024**2), "MB")

GPU 1 Total Memory: 6140.5 MB


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [9]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input_image_path = self.dataframe.iloc[idx, 0]
        final_image_path = self.dataframe.iloc[idx, 1]
        input_image = Image.open(input_image_path)
        final_image = Image.open(final_image_path)
        if self.transform:
            input_image = self.transform(input_image)
            final_image = self.transform(final_image)
        return input_image, final_image

# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((256, 448)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load Dataset
dataframe = pd.read_csv("paths.csv")
dataframe = dataframe.sample(frac=1).reset_index(drop=True)
# dataset = CustomDataset(dataframe, transform=transform)

# # DataLoader
# batch_size = 8
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) 


In [13]:
class DeblurModel(nn.Module):
    def __init__(self):
        super(DeblurModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            # nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(inplace=True),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            # nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DeblurModel().to(device)
summary(model, (3, 256, 448))  # Print summary of the model
input_tensor = torch.randn(1, 3, 256, 448).to(device)

  
output_tensor = model(input_tensor)
print("Output tensor shape:", output_tensor.shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 448]           1,792
              ReLU-2         [-1, 64, 256, 448]               0
            Conv2d-3         [-1, 64, 256, 448]          36,928
              ReLU-4         [-1, 64, 256, 448]               0
            Conv2d-5        [-1, 128, 256, 448]          73,856
              ReLU-6        [-1, 128, 256, 448]               0
            Conv2d-7        [-1, 128, 256, 448]         147,584
              ReLU-8        [-1, 128, 256, 448]               0
            Conv2d-9        [-1, 256, 256, 448]         295,168
             ReLU-10        [-1, 256, 256, 448]               0
           Conv2d-11        [-1, 256, 256, 448]         590,080
             ReLU-12        [-1, 256, 256, 448]               0
           Conv2d-13        [-1, 512, 256, 448]       1,180,160
             ReLU-14        [-1, 512, 2

In [14]:
allocated_memory = torch.cuda.memory_allocated()
print("Currently allocated GPU memory:", allocated_memory / 1024**2, "MB")

# Get peak memory usage
peak_memory = torch.cuda.max_memory_allocated()
print("Peak GPU memory usage:", peak_memory / 1024**2, "MB")

Currently allocated GPU memory: 915.49169921875 MB
Peak GPU memory usage: 2721.29345703125 MB


In [15]:
dataset = CustomDataset(dataframe, transform=transform) 
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [16]:
scaler = GradScaler()

In [21]:
%%time

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    total_loss = 0
    # for input_images, final_images in train_loader:
    for i, (input_images, final_images) in enumerate(train_loader):
        input_images = input_images.to(device)
        final_images = final_images.to(device)
        optimizer.zero_grad()

        with autocast():
            outputs = model(input_images)
            loss = criterion(outputs, final_images)
        # Forward pass
        # outputs = model(input_images)

        # Calculate the loss
        # loss = criterion(outputs, final_images)
        total_loss += loss.item()

        # Backward pass and optimization

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # loss.backward()
        # optimizer.step()
        if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, len(train_loader), loss.item()))
 
    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')

Epoch [1/1], Step [100/36000], Loss: 0.0015
Epoch [1/1], Step [200/36000], Loss: 0.0011
Epoch [1/1], Step [300/36000], Loss: 0.0008
Epoch [1/1], Step [400/36000], Loss: 0.0006
Epoch [1/1], Step [500/36000], Loss: 0.0008
Epoch [1/1], Step [600/36000], Loss: 0.0016
Epoch [1/1], Step [700/36000], Loss: 0.0025
Epoch [1/1], Step [800/36000], Loss: 0.0027
Epoch [1/1], Step [900/36000], Loss: 0.0011
Epoch [1/1], Step [1000/36000], Loss: 0.0010
Epoch [1/1], Step [1100/36000], Loss: 0.0011
Epoch [1/1], Step [1200/36000], Loss: 0.0024
Epoch [1/1], Step [1300/36000], Loss: 0.0014
Epoch [1/1], Step [1400/36000], Loss: 0.0006
Epoch [1/1], Step [1500/36000], Loss: 0.0009
Epoch [1/1], Step [1600/36000], Loss: 0.0010
Epoch [1/1], Step [1700/36000], Loss: 0.0012
Epoch [1/1], Step [1800/36000], Loss: 0.0002
Epoch [1/1], Step [1900/36000], Loss: 0.0017
Epoch [1/1], Step [2000/36000], Loss: 0.0005
Epoch [1/1], Step [2100/36000], Loss: 0.0011
Epoch [1/1], Step [2200/36000], Loss: 0.0008
Epoch [1/1], Step [

In [22]:
torch.save(model.state_dict(), 'trained_model2.pth')
print("Model saved successfully.")

Model saved successfully.
